In [1]:
#Create drive folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Get and process the dataset QMSum
Code from : https://github.com/Yale-LILY/QMSum/blob/main/data_process.ipynb

In [ ]:
import json

# read the dataset
# please enter the path of your data
split = 'val'
data_path = 'drive/My Drive/Colab Notebooks/finalproject/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 35 meetings in the val set.


In [ ]:
data[0]

In [ ]:
from nltk import word_tokenize
# tokneize a sent
def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [ ]:
# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# process data for BART
# the input of the model here is the gold span corresponding to each query
bart_data_gold = []
for i in range(len(data)):
    # get meeting content
    entire_src = []
    for k in range(len(data[i]['meeting_transcripts'])):
      cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
      cur_turn_content = tokenize(data[i]['meeting_transcripts'][k]['content'])
      if len(cur_turn_content.split()) > 5:
          cur_turn += cur_turn_content
          entire_src.append(cur_turn)
    entire_src = ' '.join(entire_src)
    for j in range(len(data[i]['general_query_list'])):
        cur = {}
        query = tokenize(data[i]['general_query_list'][j]['query'])
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + entire_src + ' </s>')
        target = tokenize(data[i]['general_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
    for j in range(len(data[i]['specific_query_list'])):
        cur = {}
        query = tokenize(data[i]['specific_query_list'][j]['query'])
        src = []
        # get the content in the gold span for each query
        for span in data[i]['specific_query_list'][j]['relevant_text_span']:
          assert len(span) == 2
          st, ed = int(span[0]), int(span[1])
          span_content = []
          for k in range(st, ed + 1):
              cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
              cur_turn_content = tokenize(data[i]['meeting_transcripts'][k]['content'])
              if len(cur_turn_content.split()) > 5:
                  cur_turn += cur_turn_content
                  span_content.append(cur_turn)
          if span_content:
              src.append(' '.join(span_content))
        src = ' '.join(src)
        cur['src'] = clean_data('<s> ' + query + ' </s> ' + src + ' </s>')
        target = tokenize(data[i]['specific_query_list'][j]['answer'])
        cur['tgt'] = target
        bart_data_gold.append(cur)
        
print('Total {} query-summary pairs in the {} set'.format(len(bart_data_gold), split))
print(bart_data_gold[2])
with open('drive/My Drive/Colab Notebooks/finalproject/processed_span_' + split + '._gold.jsonl', 'w') as f:
    for i in range(len(bart_data_gold)):
        print(json.dumps(bart_data_gold[i]), file=f)

In [ ]:
bart_data_gold[340]

In [18]:
#process the data from our locator
import json

# Path to the JSON file
file_path = 'drive/My Drive/Colab Notebooks/finalproject/final_data_ELECTRA.json'

# Load the JSON file
with open(file_path, "r") as f:
    data = json.load(f)

# Convert to a list of dictionaries
processed_data = []
for meeting, content in data.items():
    for key, value in content.items():
        if key.startswith("span"):
            index = key.replace("span", "")
            answer_key = "answer" + index
            item = {"src": value, "tgt": content.get(answer_key)}
            processed_data.append(item)

In [ ]:
data['meeting: IS1003d.json']['span1']

In [ ]:
data['meeting: IS1003d.json']['answer1']

In [19]:
len(processed_data)

244

# Get datasets from Drive

In [ ]:
# function to get the preprocessed jsonl files
import json

def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [ ]:
#get preprocessed datasets
bart_data_gold_train = read_jsonl('drive/My Drive/Colab Notebooks/finalproject/processed_span_train._gold.jsonl')
bart_data_gold_val = read_jsonl('drive/My Drive/Colab Notebooks/finalproject/processed_span_val._gold.jsonl')

print("Train set len : ", len(bart_data_gold_train))
print("Val set len : ", len(bart_data_gold_val))


Train set len :  1257
Val set len :  272


# Fine-tuning

In [ ]:
pip install transformers

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
import torch

# Load the model and tokenizer
model_name = "knkarthick/MEETING_SUMMARY"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Prepare the training data (example)
transcriptions = [d["src"] for d in bart_data_gold_train]  # List of meeting transcriptions
summaries = [d["tgt"] for d in bart_data_gold_train]  # List of meeting summaries

# Encode the data
inputs = tokenizer.batch_encode_plus(
    transcriptions,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=1024
)

labels = tokenizer.batch_encode_plus(
    summaries,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=256
)

# Extract input and output tensors
# input_ids = inputs["input_ids"]
# attention_mask = inputs["attention_mask"]
# labels = labels["input_ids"]
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
labels = labels.input_ids

# Configure training
optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

# Training
model.train()
for epoch in range(3):  # Number of epochs
    total_loss = 0
    for i in range(len(input_ids)):
        optimizer.zero_grad()
        outputs = model(input_ids[i].unsqueeze(0), attention_mask=attention_mask[i].unsqueeze(0), labels=labels[i].unsqueeze(0))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % 10 == 0:
            print(f"Iteration {i+1}/{len(input_ids)} - Loss: {loss.item()}")
    
    avg_loss = total_loss / len(input_ids)
    print(f"Epoch {epoch+1}/{5} - Average Loss: {avg_loss}")
    


In [ ]:
# Save the fine-tuned model
model.save_pretrained("drive/My Drive/Colab Notebooks/finalproject/trained_model/fine_tuned_meeting_bart")
tokenizer.save_pretrained("drive/My Drive/Colab Notebooks/finalproject/trained_model/fine_tuned_meeting_bart")

# Evaluation
In this part, we evaluate the validation set with different models

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from transformers import BartTokenizer

# Function to truncate the text to a length  max 1024 tokens
def truncate_text(text, tokenizer, max_length=1024):
    if len(tokenizer.tokenize(text)) < (max_length-3):
        #trace
        print("no truncate : " + str(len(tokenizer.tokenize(text))))

        return text
    else:
        #trace
        before_after = " " + str(len(tokenizer.tokenize(text)))

        #get the query before encoding
        # Find the index of the first and last special tokens
        start_token_idx = text.find("<s>")
        end_token_idx = text.find("</s>")

        # Extract the query between the tags
        query = text[start_token_idx+len("<s>"):end_token_idx].strip()

        
        # Remove the query part and the tags from the text
        truncated_text = text[end_token_idx+len("</s>"):].strip()

        #encode decode
        encoded = tokenizer.encode(truncated_text, truncation=True, max_length=max_length)
        truncated_text = tokenizer.decode(encoded, skip_special_tokens=True)

        #be sure the length won't be over max_length
        additional_length = len(tokenizer.tokenize("<s> " + query + " </s> " + " </s>"))
        
        if len(tokenizer.tokenize(truncated_text)) > (max_length - additional_length-3):
          truncated_tokens = tokenizer.tokenize(truncated_text)[:(max_length - additional_length-5)]
          truncated_text = tokenizer.convert_tokens_to_string(truncated_tokens)
          
        # Concatenate the query with the truncated text and add the </s> tag at the end
        truncated_text = "<s> " + query + " </s> " + truncated_text + " </s>"

        #trace
        before_after = before_after + " -> " + str(len(tokenizer.tokenize(truncated_text)))
        print(before_after)
        
        return truncated_text

In [7]:
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge import Rouge


#function to create the model
def evaluate(MODEL_NAME, val_set):

  #get model
  model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)
  tokenizer = BartTokenizer.from_pretrained(MODEL_NAME)

  #param model
  params = {
    "model": model,
    "tokenizer": tokenizer,
    "model_name_or_path": None,
    "task": "summarization",
    "framework": "pt",
    "device": 0,
    "max_length": 350,
    "min_length": 30,
    "length_penalty": 2.0,
    "num_beams": 4,
    "repetition_penalty": 1.0,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
    "temperature": 1.0,
  }

  #pipeline
  custom_pipeline = pipeline(**params)

  #get only relevant span for predicting
  inputs = [d["src"] for d in val_set]  # List of meeting transcriptions
  ref = [d["tgt"] for d in val_set] #list of reference summarizes

  #summarizes
  sums = []

  #get every sum
  for i, text in enumerate(inputs):

    #progression
    print("[---- "+str(i)+"----]")
    truncated_text = truncate_text(text,tokenizer, max_length=1024)
    print("len tokens : " + str(len(tokenizer.tokenize(truncated_text))) + " ("+str(len(truncated_text.split()))+")")

    summary = custom_pipeline(truncated_text)
    sums.append(summary[0]['summary_text'])


  return sums,ref

In [8]:
from statistics import mean
def get_rouge_l(sums, refs):

  #Get ROUGE Scores
  rouge_l_scores = []
  rouge_1_scores = []
  rouge_2_scores = []

  for i in range(len(sums)):
    hypothesis = sums[i]
    reference = refs[i]
    rouge = Rouge()
    score = rouge.get_scores([hypothesis], [reference])

    #rouge 1
    rouge_1_score = score[0]["rouge-1"]["f"]
    rouge_1_scores.append(rouge_1_score)
    #rouge 2
    rouge_2_score = score[0]["rouge-2"]["f"]
    rouge_2_scores.append(rouge_2_score)
    #rouge l
    rouge_l_score = score[0]["rouge-l"]["f"]
    rouge_l_scores.append(rouge_l_score)
  
  return mean(rouge_1_scores), mean(rouge_2_scores), mean(rouge_l_scores)

In [27]:
#Models
BART = "facebook/bart-large-cnn"
MEETING_SUM = "knkarthick/MEETING_SUMMARY"

MY_BART_CNN = "drive/My Drive/Colab Notebooks/finalproject/trained_model/fine_tuned_bart_cnn"
MY_MEETING_SUM = "drive/My Drive/Colab Notebooks/finalproject/trained_model/fine_tuned_meeting_bart"

MODEL = MY_MEETING_SUM

#get rouge-l score
sums, refs = evaluate(MODEL,processed_data)

Your max_length is set to 350, but your input_length is only 196. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=98)


[---- 0----]
no truncate : 194
len tokens : 194 (134)


Your max_length is set to 350, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


[---- 1----]
no truncate : 113
len tokens : 113 (76)


Your max_length is set to 350, but your input_length is only 153. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=76)


...

[---- 241----]
no truncate : 113
len tokens : 113 (80)


Your max_length is set to 350, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


[---- 242----]
no truncate : 78
len tokens : 78 (52)


Your max_length is set to 350, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)


[---- 243----]
no truncate : 113
len tokens : 113 (80)


In [28]:
r1, r2, rl = get_rouge_l(sums, refs)
print("r1 : " + str(r1) + " | r2 : " + str(r2) +" | rl : " + str(rl))

r1 : 0.2527887218842651 | r2 : 0.05717584641429692 | rl : 0.18633243849651931


In [ ]:
sums[2]

In [ ]:
processed_data[2]

In [ ]:
refs[1]

'to save time , speaker mn005 will only mark the sample of transcribed data for regions of overlapping speech , as opposed to marking all acoustic events . the digits extraction task will be delegated to whomever is working on acoustics for the meeting recorder project .'